In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

In [2]:
import torch
import torch
import augdataset
import loss, train_utils, models
from utils import get_features, show_images
from models import Model
import numpy as np

In [3]:
### pretrained model clustering 
# import torchvision.models as models
# import torch.nn as nn
# import torch
# import torch.nn.functional as F

# # imagenet_resnetmodel= models.resnet18().cuda()

# # imagenet_resnetmodel.load_state_dict(torch.load('models/resnet18-5c106cde.pth'))
# # imagenet_resnetmodel = torch.nn.Sequential(*list(imagenet_resnetmodel.children())[:-1])

# # fps, features = get_features(imagenet_resnetmodel, ['../digitization/data/fin_docs/pdf_images/*.*'])
# # features = [x.flatten() for x in features]
# # cluster based on features
# from sklearn.cluster import KMeans, DBSCAN
# import numpy as np
# import hdbscan
# # kmeans = KMeans(n_clusters=100, random_state=0).fit(features)
# labels = hdbscan.HDBSCAN(cluster_selection_epsilon=0.005).fit(features).labels_

# # labels = kmeans.labels_
# # the distribution of labels
# from collections import Counter
# print(Counter(labels), len(set(labels)))
# # reduce features to 2-dim
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# # pca = PCA(n_components=2)
# # features_reduced = pca.fit_transform(features)
# features_reduced = TSNE(n_components=2, learning_rate='auto',
#                   init='random').fit_transform(np.array(features))

# # plot features 
# import seaborn as sns
# import matplotlib.pyplot as plt
# # features_reduced = TSNE(n_components=2, learning_rate='auto',
# #                   init='random').fit_transform(np.array(features))
# plt.figure(figsize=(18, 12))
# sns.scatterplot(x=features_reduced[:, 0], y=features_reduced[:, 1], hue=labels, palette='gist_rainbow_r')
# # sns.displot(data=df_c, x=df_c['probability'])
# plt.show()

In [4]:
# get the dataloader with transformations - no augmentations


In [ ]:
# function to the features from images 
model = Model().cuda()
model.load_state_dict(torch.load('models/modelv2.pt')['model'])

fps, features = get_features(model, ['../digitization/data/fin_docs/pdf_images/*.*'])

/root/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/root/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Number of Images:  1394
Compose(
    Resize(size=(150, 150), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
    Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.201])
)


In [ ]:
# cluster based on features
from sklearn.cluster import KMeans, DBSCAN
import numpy as np
import hdbscan
# kmeans = KMeans(n_clusters=100, random_state=0).fit(features)
labels = hdbscan.HDBSCAN(cluster_selection_epsilon=0.005).fit(features).labels_

# labels = kmeans.labels_


In [ ]:
# number of points/images
len(labels)

In [ ]:
# the distribution of labels
from collections import Counter
Counter(labels), len(set(labels))

In [ ]:
# len(features)

In [ ]:
# features_reduced[:, 0]

In [ ]:
# reduce features to 2-dim
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
# pca = PCA(n_components=2)
# features_reduced = pca.fit_transform(features)
features_reduced = TSNE(n_components=2, learning_rate='auto',
                  init='random').fit_transform(np.array(features))

In [ ]:
# plot features 
import seaborn as sns
import matplotlib.pyplot as plt
# features_reduced = TSNE(n_components=2, learning_rate='auto',
#                   init='random').fit_transform(np.array(features))
plt.figure(figsize=(18, 12))
sns.scatterplot(x=features_reduced[:, 0], y=features_reduced[:, 1], hue=labels, palette='gist_rainbow_r')
# sns.displot(data=df_c, x=df_c['probability'])
plt.show()

In [ ]:
# zip labels with filepaths of images
preds = list(zip(labels, fps))

In [ ]:
from skimage import io
images = [io.imread(fp) for cluster, fp in preds if cluster == 15]

In [ ]:
show_images(images[:10])

In [ ]:

import shutil, os
output_dir = f'../../data_imgs/pretextselfsupervisedv4'
# output_dir = f'../../data_imgs/selfsuperviseddbscanesp0.12'

clusters_to_write = preds

try:
    os.mkdir(output_dir)
except Exception as e:
    print(e)
    shutil.rmtree(output_dir)
    os.mkdir(output_dir)
    
for cluster in list(set([cluster for cluster, _ in clusters_to_write])):
#     print(class_)
    os.mkdir(os.path.join(output_dir, str(cluster)))
    
for cluster, fp in clusters_to_write:
    cluster_dir = os.path.join(output_dir, str(cluster))
    filename = os.path.basename(fp)
    dest_fp = os.path.join(cluster_dir, f'{filename}')
    shutil.copy(fp, dest_fp)
    

In [ ]:
!7z a {os.path.abspath(os.path.dirname(output_dir))}/{os.path.basename(output_dir)}.7z {output_dir} 